In [1]:
# https://towardsdatascience.com/handwritten-digit-mnist-pytorch-977b5338e627

In [2]:
import numpy as np
import torch

import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim

In [34]:
class MNIST_Classifier:
    
    def __init__(self):
        self.model = None
        self.optimizer = None
    
    def from_file(self, filename):
        self.model = torch.load(filename)
        self.optimizer = optim.Adam(model.parameters())
        
    def to_file(self, filename):
        torch.save(self.model, filename) 
    
    def initialize(self):
        self.model = nn.Sequential(
                        nn.Linear(28 * 28, 128),
                        nn.ReLU(),
                        nn.Linear(128, 96),
                        nn.ReLU(),
                        nn.Linear(96, 32),
                        nn.ReLU(),
                        nn.Linear(32, 10),
                        nn.LogSoftmax(dim=0))
        
        self.optimizer = optim.Adam(self.model.parameters())
        
    def train(self):
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
        
        trainset = datasets.MNIST('data/MNIST/trainset', download=True, train=True, transform=transform)
        testset = datasets.MNIST('data/MNIST/testset', download=True, train=False, transform=transform)
        
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
        testloader  = torch.utils.data.DataLoader(testset,  batch_size=64, shuffle=True)
        
        epochs = 20
        criterion = nn.NLLLoss()
        
        for ep in range(epochs):
            loss_total = 0
            for images, labels in trainloader:

                images = images.view(images.shape[0], -1)
                self.optimizer.zero_grad()
                output = self.model(images)
                loss = criterion(output, labels)
                loss.backward()
                self.optimizer.step()

                loss_total += loss.item()
                
            print("Epoch {}, Loss: {}".format(ep, loss_total/len(trainloader)))
                
            if ep % 5 == 0:
                TP = 0
                TOTAL = 0
                for images, labels in testloader:
                    images = images.view(images.shape[0], -1)
                    TOTAL += len(labels)
                    for i in range(len(labels)):
                        TP += self.predict(images[i]) == labels[i]
                        
                print("Validation: {}/{} ({}%)".format(TP, TOTAL, TP*100/TOTAL))
    
    def predict(self, image):
        with torch.no_grad():
            logps = self.model(image)

        ps = torch.exp(logps)
        probab = list(ps.numpy())
        return np.argmax(probab)

In [35]:
clf = MNIST_Classifier()
clf.initialize()
clf.train()

Epoch 0, Loss: 2.353235550018262
Validation: 6811/10000 (68%)
Epoch 1, Loss: 2.13452857821735
Epoch 2, Loss: 2.0763728391132883
Epoch 3, Loss: 2.046477670608553
Epoch 4, Loss: 2.028562980165868
Epoch 5, Loss: 2.0190116146734276
Validation: 9728/10000 (97%)
Epoch 6, Loss: 2.007737347947509
Epoch 7, Loss: 2.0009683070660653
Epoch 8, Loss: 1.9934106006551144
Epoch 9, Loss: 1.9900205156950554
Epoch 10, Loss: 1.9821983338164877
Validation: 9755/10000 (97%)
Epoch 11, Loss: 1.9798042114609595
Epoch 12, Loss: 1.973317506470914
Epoch 13, Loss: 1.9727705574747343
Epoch 14, Loss: 1.9704372717627585
Epoch 15, Loss: 1.9674350196110415
Validation: 9731/10000 (97%)
Epoch 16, Loss: 1.9678448980042675
Epoch 17, Loss: 1.9661459100526024
Epoch 18, Loss: 1.961413344340538
Epoch 19, Loss: 1.959915121226931


In [36]:
clf.to_file('model/mnist_model.pt')

/home/stwasik/.local/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/stwasik/.local/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/stwasik/.local/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/stwasik/.local/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LogSoftmax. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't